## Berechnungen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabeichung für Messwerte zu berechnen.   
Zudem wird die Gieränderung in rad bestimmt. Die Ergebnisse können zurück in die Merkmal CSV Datei geschrieben werden.

In [1]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [2]:
featuresDf = pandas.read_csv("../daten/merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,Messwerte,StartZeitfenster,StartBewegungsdaten,StartBelichtung,Belichtungszeit
count,4.983200e+04,49832.000000,49832.000000,49832.000000,49832.000000,4.983200e+04,4.983200e+04,4.983200e+04,4.983200e+04
mean,1.521224e+12,53.606883,10.128745,14.058019,19.370846,7.614164e+13,7.614158e+13,7.614167e+13,1.268842e+07
std,7.273196e+07,0.027369,0.058248,3.550545,0.613684,3.221526e+13,3.221526e+13,3.221526e+13,7.765416e+06
min,1.521124e+12,53.557396,10.002181,5.004000,6.000000,2.999506e+13,2.999499e+13,2.999509e+13,9.376720e+05
25%,1.521128e+12,53.579277,10.091159,11.700000,19.000000,3.372892e+13,3.372885e+13,3.372895e+13,7.979408e+06
50%,1.521212e+12,53.613132,10.140025,14.256000,19.000000,7.307148e+13,7.307141e+13,7.307151e+13,1.000184e+07
75%,1.521297e+12,53.626503,10.162369,16.596000,20.000000,1.079068e+14,1.079067e+14,1.079068e+14,2.000368e+07
max,1.521302e+12,53.656776,10.229659,24.804000,48.000000,1.122372e+14,1.122371e+14,1.122372e+14,3.287369e+07


In [3]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49832 entries, 0 to 49831
Data columns (total 16 columns):
Zeitstempel            49832 non-null int64
Breitengrad            49832 non-null float64
Laengengrad            49832 non-null float64
Geschwindigkeit        49832 non-null float64
AccelerometerX         49832 non-null object
AccelerometerY         49832 non-null object
AccelerometerZ         49832 non-null object
Azimuth                49832 non-null object
Nick                   49832 non-null object
Roll                   49832 non-null object
SensorZeitstempel      49832 non-null object
Messwerte              49832 non-null int64
StartZeitfenster       49832 non-null int64
StartBewegungsdaten    49832 non-null int64
StartBelichtung        49832 non-null int64
Belichtungszeit        49832 non-null int64
dtypes: float64(3), int64(6), object(7)
memory usage: 6.1+ MB


In [4]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf.AccelerometerX.head()

0    0.66929 0.81137 0.75647 0.22221 -0.14389 -0.13...
1    0.2285 0.13695 0.37016 0.08172 -0.05709 0.3834...
2    -2.72057 3.72297 3.37689 -1.28232 -0.90316 0.4...
3    -0.06289 0.54739 0.6219 -1.35643 -0.54872 0.28...
4    -0.5752 -1.41006 -1.3118 -0.58965 -1.32969 0.6...
Name: AccelerometerX, dtype: object

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
AccelerometerX       non-null object   
AccelerometerY       non-null object   
AccelerometerZ       non-null object   
Azimuth              non-null object   
Nick                 non-null object   
Roll                 non-null object   
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [5]:
accXList = featuresDf.AccelerometerX.str.split(" ").tolist()

In [7]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf.AccelerometerX.str.split(" ")).values

array([[ ['0.66929', '0.81137', '0.75647', '0.22221', '-0.14389', '-0.13032', '-0.34931', '-0.26401', '-0.349', '-0.41699', '-0.27218', '-0.38619', '-0.29352', '-0.46454', '-0.47877', '-0.3829', '-0.13767', '-0.11002', '0.01936', '0.27609']],
       [ ['0.2285', '0.13695', '0.37016', '0.08172', '-0.05709', '0.38348', '-0.27536', '-0.6339', '0.21317', '-0.05919', '-0.44563', '-0.78543', '-0.52098', '-1.10619', '-0.19531', '-0.7384', '-0.60594', '-0.54593', '0.00775', '-0.26951']],
       [ ['-2.72057', '3.72297', '3.37689', '-1.28232', '-0.90316', '0.47276', '0.60817', '1.13021', '1.93092', '0.62547', '-2.50279', '-0.54644', '0.81943', '-0.67742', '-0.06682', '-0.23722', '0.45391', '1.206', '1.94558', '2.88967']],
       ..., 
       [ ['0.45461', '-0.73955', '-0.79082', '1.49724', '-1.40708', '-0.83452', '-2.42973', '-1.1623', '0.25003', '2.89688', '2.40945', '-1.6733', '-0.63377', '-1.94736', '-0.34736', '0.5036', '0.80128', '-1.97918', '-1.32284']],
       [ ['0.76333', '3.49138', '4

In [8]:
accXList[1] # zeige Liste

['0.2285',
 '0.13695',
 '0.37016',
 '0.08172',
 '-0.05709',
 '0.38348',
 '-0.27536',
 '-0.6339',
 '0.21317',
 '-0.05919',
 '-0.44563',
 '-0.78543',
 '-0.52098',
 '-1.10619',
 '-0.19531',
 '-0.7384',
 '-0.60594',
 '-0.54593',
 '0.00775',
 '-0.26951']

In [9]:
np.array(accXList[1]).dtype

dtype('<U8')

In [10]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(np.array(accXList[0]).astype(float)).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())

In [11]:
accXDf = pandas.concat(accList, ignore_index=True)

In [12]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0.66929,0.81137,0.75647,0.22221,-0.14389,-0.13032,-0.34931,-0.26401,-0.34900,-0.41699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.22850,0.13695,0.37016,0.08172,-0.05709,0.38348,-0.27536,-0.63390,0.21317,-0.05919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.72057,3.72297,3.37689,-1.28232,-0.90316,0.47276,0.60817,1.13021,1.93092,0.62547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.06289,0.54739,0.62190,-1.35643,-0.54872,0.28130,0.57759,0.60009,1.49150,1.49977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.57520,-1.41006,-1.31180,-0.58965,-1.32969,0.65253,1.15038,-0.27477,-0.61810,0.01130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
print(accXDf.head(2))
accXDf.T.mean().head(2)

        0        1        2        3        4        5        6        7   \
0  0.66929  0.81137  0.75647  0.22221 -0.14389 -0.13032 -0.34931 -0.26401   
1  0.22850  0.13695  0.37016  0.08172 -0.05709  0.38348 -0.27536 -0.63390   

        8        9        10       11       12       13       14      15  \
0 -0.34900 -0.41699 -0.27218 -0.38619 -0.29352 -0.46454 -0.47877 -0.3829   
1  0.21317 -0.05919 -0.44563 -0.78543 -0.52098 -1.10619 -0.19531 -0.7384   

        16       17       18       19 ...  28  29  30  31  32  33  34  35  36  \
0 -0.13767 -0.11002  0.01936  0.27609 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN   
1 -0.60594 -0.54593  0.00775 -0.26951 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN   

   37  38  39  40  41  42  43  44  45  46  47  
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[2 rows x 48 columns]


0   -0.071226
1   -0.240857
dtype: float64

In [14]:
accXDf.shape

(49832, 48)

In [15]:
# Die Anzahl der Messerte muss immer gleich sein, ansonsten werden fehlende Messwerte
# mit NaN gefüllt.
featuresDf.Messwerte[featuresDf.Messwerte == 22]

1472     22
7171     22
7820     22
9598     22
12467    22
14556    22
15865    22
17008    22
17787    22
18630    22
20105    22
20216    22
20777    22
20799    22
22426    22
24413    22
25007    22
26352    22
27922    22
28381    22
29771    22
30297    22
32280    22
32484    22
33507    22
34925    22
35520    22
36758    22
36794    22
37013    22
38051    22
38074    22
38104    22
38474    22
38792    22
41728    22
42276    22
42417    22
42811    22
44553    22
44891    22
45040    22
45338    22
46677    22
Name: Messwerte, dtype: int64

In [16]:
# ausführen wenn alle Daten die gleiche Ahnzahl an Messwerten haben sollen. 
# Dadurch werden Reihen gelöscht!
unwantedRows = featuresDf.Messwerte[featuresDf.Messwerte != 20]
featuresDf.drop(featuresDf.index, axis=0)

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,AccelerometerX,AccelerometerY,AccelerometerZ,Azimuth,Nick,Roll,SensorZeitstempel,Messwerte,StartZeitfenster,StartBewegungsdaten,StartBelichtung,Belichtungszeit


In [17]:
accYList = featuresDf.AccelerometerY.str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [18]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49832 entries, 0 to 49831
Data columns (total 48 columns):
0     49832 non-null float64
1     49832 non-null float64
2     49832 non-null float64
3     49832 non-null float64
4     49832 non-null float64
5     49832 non-null float64
6     49831 non-null float64
7     49831 non-null float64
8     49831 non-null float64
9     49830 non-null float64
10    49830 non-null float64
11    49830 non-null float64
12    49828 non-null float64
13    49826 non-null float64
14    49826 non-null float64
15    49825 non-null float64
16    49817 non-null float64
17    49765 non-null float64
18    48180 non-null float64
19    19562 non-null float64
20    554 non-null float64
21    59 non-null float64
22    15 non-null float64
23    5 non-null float64
24    3 non-null float64
25    3 non-null float64
26    3 non-null float64
27    3 non-null float64
28    2 non-null float64
29    2 non-null float64
30    2 non-null float64
31    2 non-null float64
32    2

In [19]:
accYDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.58300,0.48170,0.38533,-1.20873,-0.46135,-0.75219,-0.57112,0.38583,0.09411,0.19785,0.35745,0.10205,-0.28613,-0.33618,-0.34560,-0.04666,-0.77285,-0.71025,-0.27709,0.65171,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.28650,-0.61359,-0.29171,0.80856,0.43231,0.85147,-0.02369,-0.04963,0.45060,0.63627,-0.67088,-0.50608,-0.20570,0.15719,1.13704,0.89429,-0.09672,0.36697,0.69194,-0.04407,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.48539,-4.54084,-5.10369,-3.63862,-0.93426,0.06467,1.72190,4.53401,3.90300,2.21832,-1.88754,-0.39149,0.49890,-0.82675,-0.69207,-0.01738,0.66028,0.34432,0.06090,-0.57954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.26624,-1.24271,-2.05148,-3.17350,-2.00252,-0.17702,0.70112,1.87864,1.05851,1.12260,-0.55762,-0.66064,1.32553,1.22895,-0.07416,-0.02870,2.09158,7.03625,8.37177,4.75138,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.74329,-4.71651,-2.70064,3.55490,2.53744,-2.88872,-2.51021,-2.20739,-3.19096,-1.89392,1.50345,0.94225,1.64249,1.26800,0.79985,-0.72388,-1.46786,-2.07837,-1.67804,2.68746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf.AccelerometerZ.str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [21]:
# Konvertiere Nick Daten zu DataFrame
measuredDataList = featuresDf.Nick.str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [22]:
# Konvertiere Roll Daten zu DataFrame
measuredDataList = featuresDf.Roll.str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [23]:
measuredDataList = featuresDf.Azimuth.str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [24]:
measuredDataList = featuresDf.SensorZeitstempel.str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [25]:
accXMean = accXDf.T.mean()
accXMean.head(2)

0   -0.071226
1   -0.240857
dtype: float64

In [26]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['MittelX'] = accXMean
featuresDf['MittelY'] = accYDf.T.mean()
featuresDf['MittelZ'] = accZDf.T.mean()
featuresDf['MittelNick'] = nickDf.T.mean()
featuresDf['MittelRoll'] = rollDf.T.mean()

In [27]:
featuresDf.MittelX.head(2)

0   -0.071226
1   -0.240857
Name: MittelX, dtype: float64

In [28]:
featuresDf.columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartZeitfenster',
       'StartBewegungsdaten', 'StartBelichtung', 'Belichtungszeit', 'MittelX',
       'MittelY', 'MittelZ', 'MittelNick', 'MittelRoll'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [29]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [30]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [31]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [32]:
featuresDf['AzimuthAenderung'] = calculateAngelChangeAzimuth(azimuthDf)

In [33]:
# Berechne Varianz
featuresDf['VarianzX'] = calcVariance(featuresDf.MittelX,accXDf)
featuresDf['VarianzY'] = calcVariance(featuresDf.MittelY,accYDf)
featuresDf['VarianzZ'] = calcVariance(featuresDf.MittelZ,accZDf)
featuresDf['VarianzNick'] = calcVariance(featuresDf.MittelNick,nickDf)
featuresDf['VarianzRoll'] = calcVariance(featuresDf.MittelRoll,rollDf)

In [34]:
featuresDf.VarianzX.head()

0    0.15801
1    0.16816
2    2.87086
3    0.81137
4    0.84476
Name: VarianzX, dtype: float64

In [35]:
calculateStandardDeviation(featuresDf.VarianzX)

array([ 0.3975 ,  0.41007,  1.69436, ...,  1.43812,  2.25887,  1.92915])

In [36]:
# Berechne Standardabweichung
featuresDf['AbweichungX'] = calculateStandardDeviation(featuresDf.VarianzX)
featuresDf['AbweichungY'] = calculateStandardDeviation(featuresDf.VarianzY)
featuresDf['AbweichungZ'] = calculateStandardDeviation(featuresDf.VarianzZ)
featuresDf['AbweichungNick'] = calculateStandardDeviation(featuresDf.VarianzNick)
featuresDf['AbweichungRoll'] = calculateStandardDeviation(featuresDf.VarianzRoll)

In [37]:
featuresDf.AbweichungX.head()

0    0.39750
1    0.41007
2    1.69436
3    0.90076
4    0.91911
Name: AbweichungX, dtype: float64

In [38]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49832 entries, 0 to 49831
Data columns (total 48 columns):
0     49832 non-null float64
1     49832 non-null float64
2     49832 non-null float64
3     49832 non-null float64
4     49832 non-null float64
5     49832 non-null float64
6     49831 non-null float64
7     49831 non-null float64
8     49831 non-null float64
9     49830 non-null float64
10    49830 non-null float64
11    49830 non-null float64
12    49828 non-null float64
13    49826 non-null float64
14    49826 non-null float64
15    49825 non-null float64
16    49817 non-null float64
17    49765 non-null float64
18    48180 non-null float64
19    19562 non-null float64
20    554 non-null float64
21    59 non-null float64
22    15 non-null float64
23    5 non-null float64
24    3 non-null float64
25    3 non-null float64
26    3 non-null float64
27    3 non-null float64
28    2 non-null float64
29    2 non-null float64
30    2 non-null float64
31    2 non-null float64
32    2

In [39]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.66929,0.81137,0.75647,0.22221,-0.14389,-0.13032,-0.34931,-0.26401,-0.34900,-0.41699,-0.27218,-0.38619,-0.29352,-0.46454,-0.47877,-0.38290,-0.13767,-0.11002,0.01936,0.27609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.22850,0.13695,0.37016,0.08172,-0.05709,0.38348,-0.27536,-0.63390,0.21317,-0.05919,-0.44563,-0.78543,-0.52098,-1.10619,-0.19531,-0.73840,-0.60594,-0.54593,0.00775,-0.26951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.72057,3.72297,3.37689,-1.28232,-0.90316,0.47276,0.60817,1.13021,1.93092,0.62547,-2.50279,-0.54644,0.81943,-0.67742,-0.06682,-0.23722,0.45391,1.20600,1.94558,2.88967,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.06289,0.54739,0.62190,-1.35643,-0.54872,0.28130,0.57759,0.60009,1.49150,1.49977,0.58702,1.26652,2.20852,1.35321,1.08268,1.32594,1.07620,2.46997,2.02206,1.61777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.57520,-1.41006,-1.31180,-0.58965,-1.32969,0.65253,1.15038,-0.27477,-0.61810,0.01130,0.11642,0.23115,1.02780,0.96025,0.44654,0.70976,-0.67322,-1.87156,-1.45116,0.73922,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
featuresDf.MittelX.head()

0   -0.071226
1   -0.240857
2    0.512262
3    0.977270
4   -0.202993
Name: MittelX, dtype: float64

In [41]:
# Änderungen in CSV Datei schreiben
featuresDf.set_index('Zeitstempel', inplace=True)
featuresDf.to_csv('../daten/merkmale.csv')

In [42]:
pandas.read_csv("../daten/merkmale.csv").columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartZeitfenster',
       'StartBewegungsdaten', 'StartBelichtung', 'Belichtungszeit', 'MittelX',
       'MittelY', 'MittelZ', 'MittelNick', 'MittelRoll', 'AzimuthAenderung',
       'VarianzX', 'VarianzY', 'VarianzZ', 'VarianzNick', 'VarianzRoll',
       'AbweichungX', 'AbweichungY', 'AbweichungZ', 'AbweichungNick',
       'AbweichungRoll'],
      dtype='object')

In [43]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accXDf.index.names = ["index"]
accXDf.to_csv('../daten/accX.csv')
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/accY.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/accZ.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/nick.csv')
rollDf.index.names = ["index"]
rollDf.to_csv('../daten/roll.csv')
azimuthDf.index.names = ["index"]
azimuthDf.to_csv('../daten/azimuth.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/sensorTimestamps.csv')

In [44]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthNegativ (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.026s

OK
